In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import requests
import time
import json
import pickle

def dark_mode():

    # Configura o browser para dark mode
    driver.get("about:config")
    driver.find_element_by_id("warningButton").click()
    driver.find_element_by_id("about-config-search").send_keys("devtools.theme")
    driver.find_element_by_class_name("button-edit").click()
    driver.find_elements_by_xpath("//input[@type='text']")[1].send_keys("dark")
    driver.find_element_by_class_name("primary.button-save").click()

def remove_duplicado(dados):
    "Esta função cracha o arquivo json ainda não resolvido"
    lista = {}
    for l in dados:
        lista[l] = list(set(dados[l]))
    
    with open("redessocials3.json", "w") as outfile:  
        json.dump(lista, outfile)

def carrega_users():
    # Abre o banco
    with open('redessocials3.json') as f:
        data = json.load(f)   
    return data

def salva_users(listaFaltante,dados):

    for user in listaFaltante:
        dados[user[0]].append(user)

    with open("dbteste.json", "w") as outfile:  
        json.dump(dados, outfile)

def rt_users_faltantes(lista_online, dados):
    
    "Esta função retorna uma lista de usuários faltantes"
    lista_retorno = []
    for usr in lista_online:
        if usr in dados[usr[0]]:
            pass
        else:
            lista_retorno.append(usr)
    return lista_retorno


driver = webdriver.Firefox()
driver.implicitly_wait(5)

dark_mode()



In [62]:
def retorna_streamers(qtdstramers=50):

    # Esta função retorna uma lista de streamers

    qtdant = 0
    streamers = []

    driver.get("https://www.twitch.tv/directory/all?sort=VIEWER_COUNT")

    try:
        driver.find_element_by_class_name("tw-interactive.tw-link.tw-link--button").click()
    except:
        pass


    while len(streamers) < qtdstramers:

        elements = driver.find_elements_by_xpath("//a[@data-a-target='preview-card-channel-link']")

        sizeelem = len(elements)
        
        for stream in elements[qtdant:sizeelem]:    
            streamers.append(stream.get_attribute("href").replace("https://www.twitch.tv/", ""))

        qtdant = sizeelem

        elements[-1].location_once_scrolled_into_view
    

    streamers = streamers[:qtdstramers]
    return streamers


def retorna_users(streamers):
    #Retorna users
    lista_response = []
    for streamer in streamers:
        response = json.loads(requests.get(f"https://tmi.twitch.tv/group/user/{streamer}/chatters").text)
        lista_response = lista_response + response["chatters"]["viewers"]
    
    return lista_response

In [68]:
streamers = retorna_streamers(5)